In [1]:
import carla
import math
import random
import time
import numpy as np
import cv2
import time
try:
   import queue
except ImportError:
   import Queue as queue

In [2]:
client = carla.Client('localhost', 2000) # port 2000
client.set_timeout(10.0) # seconds
world = client.get_world()
settings = world.get_settings()
settings.synchronous_mode = False
# settings.fixed_delta_seconds = 0.02
world.apply_settings(settings) 

74

In [3]:
print(client.get_available_maps())
world = client.load_world('Town01')

['/Game/Carla/Maps/Town01', '/Game/Carla/Maps/Town04_Opt', '/Game/Carla/Maps/Town03_Opt', '/Game/Carla/Maps/Town02', '/Game/Carla/Maps/Town03', '/Game/Carla/Maps/Town01_Opt', '/Game/Carla/Maps/Town06_Opt', '/Game/Carla/Maps/Town05_Opt', '/Game/Carla/Maps/Town04', '/Game/Carla/Maps/Town10HD', '/Game/Carla/Maps/Town10HD_Opt', '/Game/Carla/Maps/Town07_Opt', '/Game/Carla/Maps/Town06', '/Game/Carla/Maps/Town07', '/Game/Carla/Maps/Town05', '/Game/Carla/Maps/Town02_Opt', '/Game/Carla/Maps/Town12/Town12', '/Game/Carla/Maps/Town15/Town15', '/Game/Carla/Maps/Town13/Town13', '/Game/Carla/Maps/Town11/Town11']


Spawn cars (at fixed positions):

In [4]:
blueprint_library = world.get_blueprint_library()
model3 = blueprint_library.filter('model3')[0]
ambulance = blueprint_library.filter('vehicle.ford.ambulance')[0]
spawn_point = carla.Transform(carla.Location(x=88.619987, y=101.833946, z=0.300000), carla.Rotation(pitch=0.000000, yaw=90.000046, roll=0.000000)) 
spawn_point2 = carla.Transform(carla.Location(x=88.619987, y=101.833946+12, z=0.300000), carla.Rotation(pitch=0.000000, yaw=90.000046, roll=0.000000)) 
spawn_point3 = carla.Transform(carla.Location(x=88.619987+4, y=101.833946+12, z=0.300000), carla.Rotation(pitch=0.000000, yaw=90.000046, roll=0.000000)) 

vehicle1 = world.spawn_actor(ambulance, spawn_point) # vehicle 1
vehicle2 = world.spawn_actor(model3, spawn_point2) # vehicle 2
vehicle3 = world.spawn_actor(model3, spawn_point3) # vehicle 2

spectator = world.get_spectator()
transform = carla.Transform(vehicle1.get_transform().transform(carla.Location(x=+4,z=2.5)), vehicle1.get_transform().rotation)
spectator.set_transform(transform)


Create vehicle list with vehicle type and unique identifier:

In [5]:
othertraffic_names = [] # store name + id of other traffic members
othertraffic_id = [] # store id of other traffic 
actors = world.get_actors()
vehicles = [actor for actor in actors if actor.type_id.startswith('vehicle')]
ambulance = [actor for actor in actors if actor.type_id.startswith('vehicle.ford.ambulance')]
for vehicle in vehicles:
    vehicle_info = vehicle.type_id + '_' + str(vehicle.id) # create name for vehicles with vehicle type and id
    if "ambulance" in vehicle.type_id:
        ambulance_id = vehicle.id # save id of ambulance, so you can calculate position of other traffic participants relative to this ambulance, this if statement should be changed to id of 1 particular ambulance if there are multiple ambulance
    else:
        othertraffic_names.append(vehicle_info) 
        othertraffic_id.append(vehicle.id)
    #print(vehicle.get_transform().location)
print(othertraffic_names)
print(othertraffic_id)


['vehicle.tesla.model3_199', 'vehicle.tesla.model3_198']
[199, 198]


Function to retrieve relative locations:

In [6]:
def retrieve_relative_locations(egocar_id): # maincar_id is the id of the ego car (ambulance)
    locations = np.zeros(2*len(othertraffic_id)) # store relative locations in here
    egocar = client.get_world().get_actor(egocar_id)
    egocar_location = [egocar.get_transform().location.x, egocar.get_transform().location.y] 
    i = 0
    for id in othertraffic_id:
        traffic_member = client.get_world().get_actor(id)
        locations[i] = traffic_member.get_transform().location.x - egocar_location[0] # relative x position
        i += 1
        locations[i] = traffic_member.get_transform().location.y - egocar_location[1] # relative y position
        i += 1

    return locations

In [8]:
# ticks = int(duration_seconds / world.get_settings().fixed_delta_seconds)
egocar = client.get_world().get_actor(ambulance_id)
t = 5
locations_last_x = np.zeros((t,2*len(othertraffic_id)))

# apply input to other traffic
for id in othertraffic_id:
    traffic_member = client.get_world().get_actor(id)
    traffic_member.apply_control(carla.VehicleControl(throttle=0.5, steer=0, brake=0))

start_time = time.time()
i = 1
while time.time() - start_time < 5.5:
    time_elapsed = time.time() - start_time

    # will have to change something in this cause time elapsed sometimes skips values 
    if time_elapsed == i:
        locations_last_x[i-1] = retrieve_relative_locations(ambulance_id)
        print('time [s]:', i)
        i += 1
        
    

# stop other traffic  
for id in othertraffic_id:
    traffic_member = client.get_world().get_actor(id)
    traffic_member.apply_control(carla.VehicleControl(throttle=0.0, steer=0, brake=0))

print(locations_last_x)

time [s]: 1
time [s]: 2
time [s]: 3
time [s]: 4
time [s]: 5
[[4.16307068e+00 2.28558235e+02 8.57543945e-03 2.28582405e+02]
 [4.16333008e+00 2.28558174e+02 8.17108154e-03 2.28582405e+02]
 [4.16291809e+00 2.28558174e+02 8.33129883e-03 2.28582375e+02]
 [4.16326141e+00 2.28558235e+02 7.40051270e-03 2.28582405e+02]
 [4.16378021e+00 2.28558144e+02 7.86590576e-03 2.28582314e+02]]


In [38]:
intervals = np.arange(1,t+1) # 1 to t seconds
start_time = time.time()
while time.time() - start_time < 5:
    i = 0
    a = time.time() - start_time
    if a == 2:
        print(intervals[i])
        i += 1
        

1


In [29]:
a = time.time()-start_time
print(a % 2)


0.1434621810913086


array([1, 2, 3, 4, 5])